# `yield from` 语法介绍

https://peps.python.org/pep-0380/


```Python
# EXPR 指的是任意表达式，它的值必须是一个可迭代对象
RESULT = yield from EXPR
```

In [ ]:
# 简化1

_i = iter(EXPR)  # __iter__ -> __await__
try:
    _y = _i.send(None) # prime
except StopIteration as _e: # 直接结束，一次 yield 都没遇到
    _r = _e.value
else:
    while 1: # 不遇到 StopIteration 就一直循环
        try:
            _s = yield _y # 招原样 yield 回去，并接受 send 传入的值，保存到 _s
        except GeneratorExit as _e: # 处理 close
            _i.close
            raise _e
        except BaseException as _e: # 处理其他异常
            _x = sys.exc_info()
            try:
                _y = _i.throw(*_x)
            except StopIteration as _e:
                _r = _e.value
                break
        else:
            try:
                _y = _i.send(_s)  # 接受到的值原样再 send 下去
            except StopIteration as _e:
                _r = _e.value
                break
RESULT = _r  # StopIteration 后的值就是结果

In [ ]:
# 简化2

_i = iter(EXPR)  # __iter__ -> __await__
while 1: # 不遇到 StopIteration 就一直循环
    try:
        _y = _i.send(None) # 总是 None
    except StopIteration as _e: # 直接结束，一次 yield 都没遇到
        _r = _e.value
        break
    else:
        try:
            yield _y # 照原样 yield 回去，因为传入的是 None, 因此不保存
        except GeneratorExit as _e: # 处理 close
            _i.close
            raise _e
        except BaseException as _e: # 处理其他异常
            _x = sys.exc_info()
            try:
                _y = _i.throw(*_x)
            except StopIteration as _e:
                _r = _e.value
                break
RESULT = _r  # StopIteration 后的值就是结果

In [ ]:
# 简化3

_i = iter(EXPR)  # __iter__ -> __await__
while 1: # 不遇到 StopIteration 就一直循环
    try:
        _y = _i.send(None) # 总是 None
    except StopIteration as _e: # 直接结束，一次 yield 都没遇到
        _r = _e.value
        break
    else:
        yield _y # 照原样 yield 回去，因为传入的是 None, 因此不保存

RESULT = _r  # StopIteration 后的值就是结果

# 例子

## 一个同步模式的简单任务

In [40]:
def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    big_result = big_step()

    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [41]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")
    small_result = small_step()
    print("     end small_step")
    return small_result * 1000

In [42]:
def small_step():
    print("         working on small step")
    return 123

In [43]:
# 执行任务
one_task()

begin task
     begin big_step
     begin small_step
         working on small step
     end small_step
     end big_step with 123000
end task


## 遇到阻塞了

In [44]:
import time

def small_step():
    print("         sleeping for 2 seconds")
    time.sleep(2)
    print("         working on small step")
    return 123

In [45]:
# 执行任务
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
         working on small step
     end small_step
     end big_step with 123000
end task


## 听说 `yeild` 变协程可以不阻塞

In [46]:
import time

def small_step():
    print("         sleeping for 2 seconds")
    yield time.sleep(2)
    print("         working on small step")
    return 123

In [47]:
one_task()
# 会报错，因为 yield 的函数是生成器，没法直接相乘

begin task
     begin big_step
     begin small_step
     end small_step


TypeError: unsupported operand type(s) for *: 'generator' and 'int'

## `yield` 有传染性

In [48]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")
    # small_result = small_step()
    # 因为 small_step 是一个协程，因此要这样处理
    small_coro = small_step()
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            # 处理协程返回值
            small_result = e.value
            break
        else:
            ...
    print("     end small_step")
    return small_result * 1000

In [49]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
         working on small step
     end small_step
     end big_step with 123000
end task


## 将阻塞从下游传到上游

In [50]:
def small_step():
    print("         sleeping for 2 seconds")
    # 不在这里执行 sleep，而是将阻塞从下游传递到上游
    t1 = time.time()
    yield time.sleep, 2
    t2 = time.time()
    assert t2 - t1 >= 2, "睡眠时间不足"
    print("         working on small step")
    return 123

In [51]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds


AssertionError: 睡眠时间不足

In [52]:
def big_step():
    """一个大步骤"""
    ... # 其他小步骤
    print("     begin small_step")
    # small_result = small_step()
    # 因为 small_step 是一个协程，因此要这样处理
    small_coro = small_step()
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            # 处理协程返回值
            small_result = e.value
            break
        else:
            # 将 small_coro 的值向上传递
            yield x
    print("     end small_step")
    return small_result * 1000

In [53]:
one_task()
# 由于 big_step 中使用了 yield, 因此 big_step 是一个生成器函数, 因此 one_task 函数也要做如下修改

begin task
     begin big_step
     end big_step with <generator object big_step at 0x0000017D00B065E0>
end task


In [54]:
def one_task():
    """一个任务例子"""
    print("begin task")
    ... # 其他步骤
    print("     begin big_step")

    big_coro = big_step()
    while True:
        try:
            x = big_coro.send(None)
        except StopIteration as e:
            # 处理协程返回值
            big_result = e.value
            break
        else:
            print(f"     got {x}")
            # 这里调用返回的函数
            func, args = x
            func(args)

    print(f"     end big_step with {big_result}")
    ... # 其他步骤
    print("end task")

In [55]:
one_task()

begin task
     begin big_step
     begin small_step
         sleeping for 2 seconds
     got (<built-in function sleep>, 2)
         working on small step
     end small_step
     end big_step with 123000
end task


## 阶段总结

- 协程自己并不能消除阻塞
- 协程具有传染性
- 协程通过yield把阻塞换个方式传递给了上游
- 最终阻塞仍然需要被解决

## `yield from` 来帮忙

## 阶段总结2

## `yield from` 一统江湖

## 阶段总结3

## 将任务彻底协程化

## 一个通用的任务驱动器

## 完成了整个任务的协程化改造

## 阶段总结4